In [157]:
# 看看决策树是什么

import pandas as pd
from pandas import Series,DataFrame
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_digits
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
from sklearn import tree
import graphviz

clf = DecisionTreeClassifier(criterion='entropy')
clf
# DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
#             max_features=None, max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
#             splitter='best')


DecisionTreeClassifier(criterion='entropy')

In [158]:
# Titanic 生存分析

# 数据加载
train_data = pd.read_csv('./Titanic_Data/train.csv')
test_data = pd.read_csv('./Titanic_Data/test.csv')

# 1. 数据探索
## 了解数据表的基本情况：行数、列数、每列的数据类型、数据完整度；
# print(train_data.info())
# print('='*30)
# ## 了解数据表的统计情况：总数、平均值、标准差、最小值、最大值等；
# print(train_data.describe())
# print('='*30)
# ## 查看字符串类型（非数字）的整体情况；
# print(train_data.describe(include=['O']))
# print('='*30)
# ## 查看前几行数据（默认是前 5 行）；
# print(train_data.head())
# print('='*30)
# ## 查看后几行数据（默认是后5行）
# print(train_data.tail())

# 2. 数据清洗
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
# 使用票价的均值填充票价中nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

# Cabin为船舱，从Describe中可以看出缺失值较多，训练集和测试集中缺失率达到了77%和78%,无法补齐；
# Embarked为登录港口，有少量缺失值，可以补齐
# 先观察Embarked字段的取值
# print(train_data['Embarked'].value_counts())
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)
# print(train_data['Embarked'].value_counts())

# 3. 特征选择
# 将Pclass、Sex、Age这些剩余字段作特征，放入特征向量features里
# 特征选择
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]

# 使用DictVectorizer类，处理符号化的对象

dvec = DictVectorizer(sparse=False)
train_features = dvec.fit_transform(train_features.to_dict(orient='record'))

# 4. 决策树模型
# 构造ID3决策树
clf = DecisionTreeClassifier(criterion='entropy')
# 决策树训练
clf.fit(train_features, train_labels)

# 5. 模型预测&评估
test_features=dvec.transform(test_features.to_dict(orient='record'))
# 决策树预测
pred_labels = clf.predict(test_features)
# pred_labels.to_csv('pred.csv')
print('决策树预测: \n',pred_labels)

# 导出预测值
test_data['Survived']  = pred_labels
test_data.to_excel('test_data.xlsx')

# 得到决策树准确率
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
print(u'score准确率为 %.4lf'%acc_decision_tree)

# K折交叉验证
print(u'cross_val_score准确率为 %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

# 可视化
df_data = tree.export_graphviz(clf,out_file=None)
graph = graphviz.Source(df_data)
graph.view()

决策树预测: 
 [0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 0
 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0
 0 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0
 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 0 1 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 0 0 0 1 0 0 1 0 0 0]
score准确率为 0.9820
cross_val_score准确率为 0.7801


'Source.gv.pdf'

S    644
C    168
Q     77
Name: Embarked, dtype: int64
S    646
C    168
Q     77
Name: Embarked, dtype: int64


DecisionTreeClassifier(criterion='entropy')